In [45]:
#importing libraries
import numpy as np
import pandas as pd
from datetime import datetime
import os
#keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale
from sklearn.gridseach import GridSearchCV
from sklearn.model_selection import train_test_split 
from keras.wrappers.scikit_learn import KerasClassifier

#################TRAINING PARAM
verbose_level=0


from sklearn.pipeline import Pipeline
#from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Input, Dropout
from keras import Sequential
from keras import regularizers
from keras.layers import Dropout

# for combining all the hyper-parameters
import itertools 
from time import time
from functools import reduce 
import operator

def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))
from tensorflow.python.client import device_lib #If training machine has gpu limit to one core(GridSearchCv does not support multithreading and gpu use) else use all available cores
tmp=device_lib.list_local_devices()
counter=0
jobs=-1
for i in device_lib.list_local_devices():
    if(i.device_type=="GPU"):
        counter+=1
if(counter!=0):
    jobs=1
#jobs=1 #due to the low dimension of the dataset the single gpu version is slower, so I'm forcing it to the multi core version


ModuleNotFoundError: No module named 'sklearn.gridseach'

In [21]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [12]:
# Grid search for the best parameter
g_eta = [0.1,0.2,0.3]
g_batchSize = [10,20,25,30]
g_hiddeLayerunit1 = [4,8,10]
g_momentum = [0.6,0.8,0.9]
g_afHiddenLayerunit1 = ["relu", "tanh"]


In [13]:
# reading training and testing data from the csv file
trainDf = pd.read_csv ('data/csv/monks-3Train.csv',header=None).to_numpy()
testDf = pd.read_csv ('data/csv/monks-3Test.csv',header=None).to_numpy()
trainDf = minmax_scale(trainDf, feature_range=(0,1), axis=0)
testDf = minmax_scale(trainDf, feature_range=(0,1), axis=0)

In [14]:
#Preparing trainDf for training
trainX = trainDf[:, 1:7]
trainY = trainDf[:, 0]
#trainX = scaler.fit_transform(trainX)
#trainY = scaler.fit_transform(trainY)

In [15]:
#Preparing testDf for validation
testX = testDf[:, 1:7]
testY = testDf[:, 0]


In [16]:
# All combination of the hyper-paramers
gridParameters=[g_eta,g_batchSize,g_hiddeLayerunit1,g_momentum,g_afHiddenLayerunit1]
allcombination = list(itertools.product(*gridParameters))


From the grid search I was able to get the best hyperparameters
eta = 0.1
batch size = 10
hiddenLayer = 10
momentum = 0.7
hiddenLayer af = relu

(0.1, 10, 10, 0.9, 'relu') - Then added a new hidden layer with activation unit of tanh and get accuracy of 99%

In [17]:
import keras as ks
tensorboard_callback = ks.callbacks.TensorBoard(log_dir="./log")


In [44]:
%%time 
def create_model(lr=0.1,momentum=0.9,g_hiddeLayerunit1=4,g_hiddeLayerunit2=4,activation1="tanh",activation2="tanh",activation3="sigmoid",init_mode='uniform', g_decay=0.01):
    model = Sequential()
    model.add(Dense(g_hiddeLayerunit1, input_dim=6, kernel_initializer=init_mode, activation=activation1))
    model.add(Dropout(0.4)),
    model.add(Dense(g_hiddeLayerunit2, kernel_initializer=init_mode, activation=activation2,kernel_regularizer=regularizers.l1(0.01))) # l1 regularization parameter could also be something we can also grid search for.
    model.add(Dropout(0.4)),
    model.add(Dense(1, activation3))
    sgd = SGD(lr=lr, momentum=momentum, decay=g_decay, nesterov=False,)  # We can add decay to hyper parameter list to get optimum value. 
    model.compile(optimizer=sgd, loss='mean_squared_error',metrics=['accuracy'])
    return model


class KerasRegressorTB(KerasClassifier):

    def __init__(self, *args, **kwargs):
        super(KerasClassifier, self).__init__(*args, **kwargs)

    def fit(self, x, y, log_dir=None, **kwargs):
        cbs = None
        if log_dir is not None:
            # Make sure the base log directory exists
            try:
                os.makedirs(log_dir)
            except OSError:
                pass
            params = self.get_params()
            conf = ",".join("{}={}".format(k, params[k])
                            for k in sorted(params))
            conf_dir_base = os.path.join(log_dir, conf)
            # Find a new directory to place the logs
            for i in itertools.count():
                try:
                    conf_dir = "{}_split-{}".format(conf_dir_base, i)
                    os.makedirs(conf_dir)
                    break
                except OSError:
                    pass
            cbs = [ks.callbacks.TensorBoard(log_dir=conf_dir, histogram_freq=0,
                               write_graph=True, write_images=False)]
        super(KerasRegressorTB, self).fit(x, y, callbacks=cbs, **kwargs)

        

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
g_hiddeLayerunit1=[4,8,16]
g_hiddeLayerunit2=[4,8,16]
g_decay = [0.01] # more estimations can be added
activation1 = ["relu", "tanh","sigmoid","softmax"]
activation2 = ["relu", "tanh","sigmoid","softmax"]
activation3 = ["relu", "tanh","sigmoid","softmax"]
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
lr=np.arange(0.1, 0.9, 0.1).tolist()
momentum=np.arange(0.1, 0.9, 0.1).tolist()
param_grid = dict(
    lr=lr,
   # g_hiddeLayerunit1=g_hiddeLayerunit1,
   # g_hiddeLayerunit2=g_hiddeLayerunit2,
   # activation1=activation1,
   # activation2=activation2,
   # activation3=activation3,
   # batch_size=batch_size,
   # init_mode=init_mode,
   # g_decay = g_decay,
   # epochs=epochs
    )
model = KerasRegressorTB(build_fn=create_model)
print("HE")


print("HEHE")
start_time = timer(None) # timing starts from this point for "start_time" variable
grid = GridSearchCV(estimator=model, param_grid=param_grid,fit_params={'log_dir': './Graph'},n_jobs=1,cv=3,verbose=verbose_level)


print("HEHEHE")
grid_result = grid.fit(trainX, trainY)
timer(start_time) # timing ends here for "start_time" variable
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print("Mean\tSTD\tParams")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


HE
HEHE


TypeError: __init__() got an unexpected keyword argument 'fit_params'

In [19]:
%reload_ext tensorboard
%tensorboard --logdir logs


Reusing TensorBoard on port 6006 (pid 12088), started 1:39:20 ago. (Use '!kill 12088' to kill it.)